In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time

#list api keys here
from config import api_key

state = 'Oaklahoma'
state_code = 40

In [35]:
url = 'http://api.census.gov/data/2010/sf1?get=P0010001,NAME&for=county:*&in=state:40&key=' + api_key
response = requests.get(url).json()
county_name = []
county_code = []
census_variable_code = []
counter = 0;
for county in response:
    if (counter > 0):
        county_name.append(county[1])
        county_code.append(county[3])
        census_variable_code.append(county[0])
    counter = counter + 1
    
all_counties = {"County Name" : county_name,
               "County Code" : county_code,
               "Census Code" : census_variable_code}

all_counties_df = pd.DataFrame(all_counties)
all_counties_df.head()

,County Name,County Code,Census Code
0,Adair County,001,22683
1,Alfalfa County,003,5642
2,Atoka County,005,14182
3,Beaver County,007,5636
4,Beckham County,009,22119


In [37]:
#url = 'http://api.census.gov/data/2010/sf1?get=P0010001,P0040003&for=zip+code+tabulation+area:*&in=state:40&key=' + api_key
#response = requests.get(url).json()
#print (response)